In [25]:
# CELL 1 – ZORLA TEMİZ VE ÇALIŞAN PYTRENDS KUR
import subprocess
import sys

# Eski pytrends’i tamamen siliyor
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "pytrends", "-y"])

# GitHub’dan direkt çalışan son sürümü kuruyor (2025’te test edildi)
subprocess.check_call([
    sys.executable, "-m", "pip", "install", 
    "git+https://github.com/GeneralMills/pytrends.git", "--force-reinstall", "--no-cache-dir"
])

print("pytrends başarıyla güncellendi! Artık hata vermez.")

Found existing installation: pytrends 4.9.2
Uninstalling pytrends-4.9.2:
  Successfully uninstalled pytrends-4.9.2
  Cloning https://github.com/GeneralMills/pytrends.git to /tmp/pip-req-build-rfrmc2_g


  Running command git clone --filter=blob:none --quiet https://github.com/GeneralMills/pytrends.git /tmp/pip-req-build-rfrmc2_g


  Resolved https://github.com/GeneralMills/pytrends.git to commit a9984ffdc9b31d853dde2ab614a77ecbf2bf33a1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.6 MB/s  0:00:07 eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 4.1 MB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 7.6 MB/s  0:00:00 eta 0:00:01
  Created wheel for pytrends: filename=pytrends-4.9.2-py3-none-any.whl size=15536 sha256=f1c92016e4977e09d1fc07819c3cfb3aa51aca0ff7c9887be34e26a14c276a37
  Stored in directory: /tmp/pip-ephem-wheel-cache-yv42uekc/wheels/3b/b3/39/9a5ca890574f7357e020a069c29f29d7923b69805a16f36b89
Successfully built pytr

In [1]:
# CELL 2 – Kütüphaneler
import json, time, pandas as pd
from pytrends.request import TrendReq
from tqdm.notebook import tqdm

pytrends = TrendReq(hl='en-US', tz=360)
print("Bağlantı OK")

Bağlantı OK


In [2]:
# CELL 3 – JSON yükle
with open('keywords.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

total = sum(len(sg['queries']) for c in data['categories'] for sg in c['sub_groups'])
print(f"{total} kelime işlenecek\n")

190 kelime işlenecek



In [3]:
# CELL 4 – En basit çalışan fonksiyon
def get_id(kw):
    try:
        s = pytrends.suggestions(kw)
        if not s: return None
        for x in s:
            if x['type'] == 'Topic': 
                return x['mid']
        return s[0]['mid']  # fallback
    except:
        return None

In [4]:

# CELL 5 – Ana döngü (10-12 dk sürer)
updated = 0
failed = []

for cat in tqdm(data['categories'], desc="Kategoriler"):
    for sg in cat['sub_groups']:
        for q in sg['queries']:
            label = q['label']
            if q.get('topic_id', '').startswith(('/m/', '/g/')):
                continue  # zaten doluysa geç
            
            tid = get_id(label)
            if tid:
                q['topic_id'] = tid
                updated += 1
                print(f"{label:30} → {tid}")
            else:
                q['topic_id'] = 'NOT_FOUND'
                failed.append(label)
            time.sleep(1.3)

print(f"\nBitti! {updated} tane dolduruldu, {len(failed)} tane bulunamadı")

Kategoriler:   0%|          | 0/9 [00:00<?, ?it/s]

Crypto Wallet                  → /g/11tskkw5c9
Memecoin                       → /g/11sfdkgmfn
Binance                        → /g/11hcz1r4wl
Web3                           → /g/11pcs9ny8w
Real Estate                    → /g/11bw1zmnts
Mortgage                       → /m/0273t5w
Housing Market                 → /g/1234z6p9
Property Tax                   → /m/020shm
Housing Bubble                 → /g/11gt__kf44
Inflation                      → /m/09jx2
Cost of Living                 → /m/01fss9
Price Hike                     → /g/11mcmf67yl
Shrinkflation                  → /m/010w1kk1
Recession                      → /m/06bmj
Recession 2025                 → /g/11mrqfmb3k
Layoff                         → /m/02jnsq
Job Cuts                       → /g/11vyhs52bj
Economic Crisis                → /g/1211cg58
Interest Rate                  → /m/04n7dpf
Mortgage Rate                  → /g/11fb0vw2fn
Credit Card Debt               → /m/05457c
Hermès Birkin                  → /g/11y119nb71
Role

In [5]:
# CELL 6 – Kaydet
with open('keywords_FINAL_2025.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("DOSYA HAZIR: keywords_FINAL_2025.json")
if failed:
    print("Bulunamayanlar:", failed)

DOSYA HAZIR: keywords_FINAL_2025.json
